#Training StyleGAN2 on Google Colab
Our first test of setting up an Generative Neural Network for creating Architectural Ornaments of different Architectural Styles and Epochs.

Original Notebook here:
'https://colab.research.google.com/github/dvschultz/ai/blob/master/StyleGAN2_Colab_Train.ipynb#scrollTo=8QrOVqEHaipA'

##Mounting Google Drive
Installation of the StyleGAN2 repository of Derrick Schultz (https://github.com/dvschultz/stylegan2) directly into Google Drive. This will make the setup a little easier, but its a little strange I admit.

First, connect your Drive to Colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Install the repo
**Only do this for the first time ever setting this up!**

If this is your first time ever running this notebook, you’ll want to install my fork of StyleGAN2 to your Drive account. Make sure you have ample space on your Drive (I’d say at least 50GB). This will install the repo and add some dependecies (like transferring from FFHQ the first time).

Every time after your first use of this notebook you’ll want to skip this cell and run the cell after this.

In [2]:
## #####SKIP this if you already have a stylegan2 folder in your google drive
%cd /content/drive/MyDrive/MRAC TERM II/01_TERM II - GROUP WORK FOLDER/HARDWARE II - GROUP WORK FOLDER/03_Colab_Repository/05_GAN2_Columns

/content/drive/MyDrive/MRAC TERM II/01_TERM II - GROUP WORK FOLDER/HARDWARE II - GROUP WORK FOLDER/03_Colab_Repository/05_GAN2_Columns


In [ ]:
## #####SKIP this if you already have a stylegan2 folder in your google drive
# !git clone https://github.com/dvschultz/stylegan2
# %cd /stylegan2
# !mkdir pkl
# %cd pkl
# !gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
# %cd ../
# !mkdir results
# !mkdir results/00001-pretrained
# %cd results/00001-pretrained
# !gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
# !mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
# %cd ../../
# %mkdir datasets

##Picking up from a previous session
If you already have the StyleGAN2 repo installed in Drive skip the above cell and run the following. This will make sure you have the latest version in case I make updates.

In [ ]:
# USE this if you already have a stylegan2 folder in google drive
%cd /content/drive/MyDrive/MRAC TERM II/01_TERM II - GROUP WORK FOLDER/HARDWARE II - GROUP WORK FOLDER/03_Colab_Repository/05_GAN2_Columns/
!git pull ####uncommend if you want to check for updates in the original StyleGAN2 repo

/content/drive/MyDrive/MRAC TERM II/01_TERM II - GROUP WORK FOLDER/HARDWARE II - GROUP WORK FOLDER/03_Colab_Repository/05_GAN2_Columns
Already up to date.


##Make sure Tensorflow 1.15 is set
Colab now defaults to Tensorflow 2. Make sure you run this cell to reset it to TF1.

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow
print(tensorflow.__version__)

1.15.2


## Listing Devices
If you are curious to know the devices used during the execution of your notebook in the cloud, try the following code −

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18328531399019999436, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7183015443251216918
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1848788480686745461
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15964005991
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6190020904817456344
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

## Preparing Dataset

Using the 'convert' command of th Imagemagick library to batch rename, resize and saving the prepared PNG's into a new folder.


In [ ]:
%cd /content/drive/MyDrive/MRAC TERM II/01_TERM II - GROUP WORK FOLDER/HARDWARE II - GROUP WORK FOLDER/03_Colab_Repository/05_GAN2_Columns/trainingset/Columns_512
# !mogrify 01.png -resize 1024x1024 -gravity center -extent 1024x1024 -background transparent -density 72 -quality 90 -path ./Columns_1024/%05d.png

/content/drive/MyDrive/MRAC TERM II/01_TERM II - GROUP WORK FOLDER/HARDWARE II - GROUP WORK FOLDER/03_Colab_Repository/03_GAN-Columns/traingset/Columns_PNG


##Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). 

I’ve seen some recommendations to run this command every time you restart your Colab machine. I think if you ahve a small-ish dataset (< 2000 images) that’s probably unnecessary.

I recommend you upload your dataset to Google Drive and copy its path from the Drive folder in Colab and paste its path in the below cell.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.


Loaction of saving the output:
./datasets/ ...

Location of Training Images:
./trainingsets/ ...

Run:
`!python dataset_tool.py create_from_images ./datasets/khmer_128 ./trainingsets/khmer_128`



In [4]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
!python dataset_tool.py create_from_images ./datasets/Columns_Black_512 ./trainingset/Columns_Black_512

Loading images from "./trainingset/Columns_Black_512"
  0% 0/222 [00:00<?, ?it/s]dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
100% 222/222 [01:49<00:00,  2.03it/s]


##Training
Note: this will require you to restart your Colab machine every 8–16 hours. You’ve been warned!

This library is set up to automatically find your latest .pkl file so it should keep re-training and making progress.

##Training Options
`--dataset`

This should be the name you used in the first path when converting your dataset.

`--mirror-augment`

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.

`--metrics`

METRICS DON’T MATTER. It’s art! Use your eyes. Set `--metrics=None` and live your life.

If you must use metrics, you have a few options. `fid50k`, the default, uses Frechet Inception Distance score. It’s what was used in StyleGAN1 and what most people know. It’s fine for images of animals and things, but it’s not great. `ppl_wend` is what StyleGAN2 prefers and claims to be more accurate. There are a bunch of other options but I’d recommend you stick with those. Note that both of these take 30–45minutes to run every time it runs so that cuts into your training time in Colab.

Run:
`!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=khmer_128 --mirror-augment=true --metrics=None`

In [7]:
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-e --dataset=Columns_512 --mirror-augment=true --metrics=None

Local submit - run_dir: results/00059-stylegan2-Columns_512-1gpu-config-e
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x55c3a56ae000 @  0x7f17a2b4d001 0x7f179f63154f 0x7f179f681b58 0x7f179f685b17 0x7f179f724203 0x55c39ef730e4 0x55c39ef72de0 0x55c39efe76f5 0x55c39efe1e0d 0x55c39ef7502c 0x55c39efb5d39 0x55c39efb2c84 0x55c39ef75231 0x55c39efe41e6 0x55c39efe1b0e 0x55c39eeb3e2b 0x55c39efe41e6 0x55c39efe1b0e 0x55c39eeb3e2b 0x55c39efe41e6 0x55c39ef7469a 0x55c39efe2a45 0x55c39ef7469a 0x55c39efe2c9e 0x55c39efe1b0e 0x55c39eeb3e2b 0x55c39efe41e6 0x55c39efe1b0e 0x55c39eeb3e2b 0x55c39efe41e6 0x55c39ef7469a
tcmalloc: large alloc 4294967296 bytes == 0x55c4a56ae000 @  0x7f17a2b4b1e7 0x7f179f63146e 0x7f179f681c7b 0x7f179f68235f 0x7f179f724103 0x55c39ef730e4 0x55c39ef72de0 0x55c39efe76f5 0x55c39efe1b0e 0x55c39ef7477a 0x55c39efe386a 0x55c39efe1b0e 0x55c39ef7477a 0x55c39efe386a 0x5

Once running, your training files will show up in the results folder.

#Testing the model (generating images)

The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

Run:
`!python run_generator.py generate-images --network=./results/00004-stylegan2-khmer_128-1gpu-config-f/network-snapshot-010192.pkl --seeds=3875451-3876000 --truncation-psi=0.7`

If Error -opensimplex not found-:
`!pip install opensimplex` in front of python run_generator.

In [8]:
!pip install opensimplex

In [10]:
!python run_generator.py generate-images --network=./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl --seeds=34623-44653 --truncation-psi=0

Local submit - run_dir: results/00059-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for seed 34623 (1/10031) ...
Generating image for seed 34624 (2/10031) ...
Generating image for seed 34625 (3/10031) ...
Generating image for seed 34626 (4/10031) ...
Generating image for seed 34627 (5/10031) ...
Generating image for seed 34628 (6/10031) ...
Generating image for seed 34629 (7/10031) ...
Generating image for seed 34630 (8/10031) ...
Generating image for seed 34631 (9/10031) ...
Generating image for seed 34632 (10/10031) ...
Generating image for seed 34633 (11/10031) ...
Generating image for seed 34634 (12/10031) ...
Generating image for seed 34635 (13/10031) ...
Generatin

In [ ]:
!pip uninstall keras
!pip install keras==2.2.4

In [ ]:
!python run_projector.py project-generated-images --network=./networks/stylegan2-Columns-config-e.pkl --seeds=0,1,5

Let’s zip the generated files and download them.

In [ ]:
# !zip -r generated-0.7.zip /content/stylegan2/results/00000-generate-images

##Near neighbors
**Added by popular demand**

Let’s say you have a seed you like, but want to see other images like it to see if there’s something better. Now you can with the `near-neighbor` argument.

### Options
`--network`, `--seeds`, and `--truncation_psi` work the same as above.

`--diameter`: this sets how far away from the seed you want to generate images. `.0000001` is really close, `.5` is reallly far.

`--num_samples`: how many samples you want to produce

`--save_vector`: this will save the vector as a file in the .npy format. You can then use this for interpolation (not super well supported right now, but can be used manually—see the Projection code for an example of reading a .npy file and interpolating it).




In [13]:
!python run_generator.py generate-neighbors --network=./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl --seeds=10 --truncation-psi=0.5 --num_samples=10 --diameter=.000000005 --save_vector

Local submit - run_dir: results/00059-generate-neighbors
dnnlib: Running run_generator.generate_neighbors() on localhost...
Loading networks from "./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for seed 10 (1/1) ...
dnnlib: Finished run_generator.generate_neighbors() in 13s.


# Truncation
Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. Most people choose between 0.5 and 1.0, but technically its infinite. 

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

###Options 
`--network`: Again, point this to your .pkl file.

`--seed`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

`!python run_generator.py truncation-traversal --network=./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl --seed=18 --start=-2.0 --stop=2.0 --increment=0.03`

In [20]:
!python run_generator.py truncation-traversal --network=./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl --seed=3875954 --start=-2.0 --stop=2.0 --increment=0.03

Local submit - run_dir: results/00065-truncation-traversal
dnnlib: Running run_generator.truncation_traversal() on localhost...
Loading networks from "./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating truncation -2.00
Generating truncation -1.97
Generating truncation -1.94
Generating truncation -1.91
Generating truncation -1.88
Generating truncation -1.85
Generating truncation -1.82
Generating truncation -1.79
Generating truncation -1.76
Generating truncation -1.73
Generating truncation -1.70
Generating truncation -1.67
Generating truncation -1.64
Generating truncation -1.61
Generating truncation -1.58
Generating truncation -1.55
Generating truncation -1.52
Generating truncation -1.49
Generating truncation -1.46
Generating truncation -1.43
Generating truncation -1.40
Generating tr

## Noise Loop Interpolation

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

Seeds aren’t used here but you do need to set one for 🌟reasons🌟. Just leave it at zero but know that changing this value won’t do much.

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--start_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

In [15]:
!python run_generator.py generate-latent-walk --network=./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl --walk-type='noiseloop' --frames 100 --seeds=0 --truncation-psi=0.5 --diameter=1.5 --start_seed=10

Local submit - run_dir: results/00061-generate-latent-walk
dnnlib: Running run_generator.generate_latent_walk() on localhost...
Loading networks from "./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for step 0/100 ...
Generating image for step 1/100 ...
Generating image for step 2/100 ...
Generating image for step 3/100 ...
Generating image for step 4/100 ...
Generating image for step 5/100 ...
Generating image for step 6/100 ...
Generating image for step 7/100 ...
Generating image for step 8/100 ...
Generating image for step 9/100 ...
Generating image for step 10/100 ...
Generating image for step 11/100 ...
Generating image for step 12/100 ...
Generating image for step 13/100 ...
Generating image for step 14/100 ...
Generating image for step 15/100 ...
Generating image fo

# Mixing Seeds

Since seeds are just points in a vector space, we can do math things to them, like adding them together. You could do this thru linear interpolation and using the middle frame, but if you want to visualize a number of options here’s a simple script to do it. This takes a number of seeds to produce a grid showing what happens when you add the row and column together (this will make more sense after running it).

`!python run_generator.py style-mixing-example --network=/content/network.pkl --row-seeds=85,100,75,458,1500 --col-seeds=55,821,1789,293 --truncation-psi=0.5`

In [19]:
!python run_generator.py style-mixing-example --network=./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl --row-seeds=85,100,75,458,1500 --col-seeds=2,300,467,658854,293 --truncation-psi=0.8

Local submit - run_dir: results/00064-style-mixing-example
dnnlib: Running run_generator.style_mixing_example() on localhost...
Loading networks from "./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating W vectors...
Generating images...
Generating style-mixed images...
Saving images...
Saving image grid...
dnnlib: Finished run_generator.style_mixing_example() in 17s.


##Interpolation


In [ ]:
!pip install opensimplex

Seeds:  `--seeds=3,11,17,25,3`

In [ ]:
# generate some random seeds
import numpy as np
import matplotlib.pyplot as plt


seeds = np.random.randint(10000000, size=9)
print(seeds)

# show the seeds
imshow(createImageGrid(generate_images_from_seeds(seeds, 0.7), 0.7 , 3))

In [21]:
!python run_generator.py generate-latent-walk --network=./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl --seeds=2715331,5438058,6083404,6611691,4373347,97727732,608078,2600519,2184699 --frames 1000 --truncation-psi=0.7

Local submit - run_dir: results/00066-generate-latent-walk
dnnlib: Running run_generator.generate_latent_walk() on localhost...
Loading networks from "./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
not enough values to generate walk
Generating image for step 0/1008 ...
Generating image for step 1/1008 ...
Generating image for step 2/1008 ...
Generating image for step 3/1008 ...
Generating image for step 4/1008 ...
Generating image for step 5/1008 ...
Generating image for step 6/1008 ...
Generating image for step 7/1008 ...
Generating image for step 8/1008 ...
Generating image for step 9/1008 ...
Generating image for step 10/1008 ...
Generating image for step 11/1008 ...
Generating image for step 12/1008 ...
Generating image for step 13/1008 ...
Generating image for step 14/1008 ...
Genera

In [22]:
#convert to video 
!ffmpeg -r 24 -i ./results/00066-generate-latent-walk/frame%5d.png -vcodec libx264 -pix_fmt yuv420p ./video/column-v2.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

## Noise Loop Interpolation

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

Seeds aren’t used here but you do need to set one. Just leave it at zero but know that changing this value won’t do much.

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--start_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

In [23]:
!python run_generator.py generate-latent-walk --network=./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl --walk-type='noiseloop' --frames 100 --seeds=0 --truncation-psi=0.5 --diameter=1.5 --start_seed=10

Local submit - run_dir: results/00067-generate-latent-walk
dnnlib: Running run_generator.generate_latent_walk() on localhost...
Loading networks from "./results/00054-stylegan2-Columns_512-1gpu-config-e/network-snapshot-006515.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for step 0/100 ...
Generating image for step 1/100 ...
Generating image for step 2/100 ...
Generating image for step 3/100 ...
Generating image for step 4/100 ...
Generating image for step 5/100 ...
Generating image for step 6/100 ...
Generating image for step 7/100 ...
Generating image for step 8/100 ...
Generating image for step 9/100 ...
Generating image for step 10/100 ...
Generating image for step 11/100 ...
Generating image for step 12/100 ...
Generating image for step 13/100 ...
Generating image for step 14/100 ...
Generating image for step 15/100 ...
Generating image fo

# Prevent Auto-Disconnect

To prevent Auto-Disconnect because of inactivity, just open the Developer Tools of Google Chrome (make shure you are working in Google Chrome Browser) and put into the console: 

`function ClickConnect(){`

`console.log("Working");`

`document.querySelector("colab-toolbar-button").click()`

`}setInterval(ClickConnect,60000)`